**Задание**

**Цель:**

Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.

**Описание задания:**

Решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с [сайта](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction). Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

**Этапы работы:**

1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

2. Подготовьте датасет к обучению моделей:

  a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.

3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

4. Обучите модель логистической регрессии с параметрами по умолчанию.

5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:

cross_validate(…, cv=10, scoring=['accuracy','recall','precision','f1'])

6. Оптимизируйте 3-4 параметра модели:

  a) Используйте GridSearchCV.

  b) Используйте RandomizedSearchCV.

  c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.

  d) Повторите п. 5 после каждого итогового изменения параметров.

7. Сформулируйте выводы по проделанной работе:

  a) Сравните метрики построенных моделей.

  b) *Сравните с полученными результатами в задании по теме «Ансамблирование».

Результат:

Получены знания по оптимизации параметров.

**1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [ ]:
data.describe(include='all')

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,918.000000,918,918,918.000000,918.000000,918.000000,918,918.000000,918,918.000000,918,918.000000
unique,NaN,2,4,NaN,NaN,NaN,3,NaN,2,NaN,3,NaN
top,NaN,M,ASY,NaN,NaN,NaN,Normal,NaN,N,NaN,Flat,NaN
freq,NaN,725,496,NaN,NaN,NaN,552,NaN,547,NaN,460,NaN
mean,53.510893,NaN,NaN,132.396514,198.799564,0.233115,NaN,136.809368,NaN,0.887364,NaN,0.553377
std,9.432617,NaN,NaN,18.514154,109.384145,0.423046,NaN,25.460334,NaN,1.066570,NaN,0.497414
min,28.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,60.000000,NaN,-2.600000,NaN,0.000000
25%,47.000000,NaN,NaN,120.000000,173.250000,0.000000,NaN,120.000000,NaN,0.000000,NaN,0.000000
50%,54.000000,NaN,NaN,130.000000,223.000000,0.000000,NaN,138.000000,NaN,0.600000,NaN,1.000000
75%,60.000000,NaN,NaN,140.000000,267.000000,0.000000,NaN,156.000000,NaN,1.500000,NaN,1.000000


**2. Подготовьте датасет к обучению моделей:**

  **a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.**

In [ ]:
# Уникальных значений каждого категориального признака немного, применим к ним get_dummies:
data_num = pd.get_dummies(data)
data_num.shape

(918, 21)

In [ ]:
# Массив с признаками и серия с целевой переменной:
df_X = data_num.drop(columns='HeartDisease')
y = data_num.HeartDisease
df_X.shape, y.shape

((918, 20), (918,))

Выполним стандартизацию значений признаков с помощью StandardScaler().

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
scaler.fit(df_X)

StandardScaler()

In [ ]:
# Датафрейм с масштабированными признаками:
X = pd.DataFrame(scaler.transform(df_X), columns=df_X.columns)

In [ ]:
X[:2]

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,-1.433140,0.410909,0.825070,-0.551341,1.382928,-0.832432,-0.515952,0.515952,-1.084138,2.075177,-0.532838,-0.229679,-0.507478,0.814275,-0.490449,0.823556,-0.823556,-0.271448,-1.002181,1.150674
1,-0.478484,1.491752,-0.171961,-0.551341,0.754157,0.105664,1.938163,-1.938163,-1.084138,-0.481887,1.876744,-0.229679,-0.507478,0.814275,-0.490449,0.823556,-0.823556,-0.271448,0.997824,-0.869056


In [ ]:
X.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
count,9.180000e+02,9.180000e+02,918.000000,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,918.000000,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02,9.180000e+02
mean,-1.083616e-16,1.954380e-16,0.000000,-3.096047e-17,4.953675e-16,1.238419e-16,7.740117e-18,-7.740117e-18,-1.741526e-16,0.000000,-6.966105e-17,1.548023e-17,-3.096047e-17,6.192094e-17,-3.096047e-17,1.161018e-16,-3.870058e-18,-1.548023e-17,1.315820e-16,2.322035e-17
std,1.000545e+00,1.000545e+00,1.000545,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00,1.000545,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00,1.000545e+00
min,-2.706015e+00,-7.154995e+00,-1.818435,-5.513413e-01,-3.018469e+00,-3.271482e+00,-5.159524e-01,-1.938163e+00,-1.084138e+00,-0.481887,-5.328378e-01,-2.296787e-01,-5.074783e-01,-1.228087e+00,-4.904493e-01,-1.214246e+00,-8.235563e-01,-2.714484e-01,-1.002181e+00,-8.690559e-01
25%,-6.906294e-01,-6.699346e-01,-0.233704,-5.513413e-01,-6.605778e-01,-8.324324e-01,-5.159524e-01,5.159524e-01,-1.084138e+00,-0.481887,-5.328378e-01,-2.296787e-01,-5.074783e-01,-1.228087e+00,-4.904493e-01,-1.214246e+00,-8.235563e-01,-2.714484e-01,-1.002181e+00,-8.690559e-01
50%,5.188098e-02,-1.295128e-01,0.221363,-5.513413e-01,4.678968e-02,-2.695748e-01,-5.159524e-01,5.159524e-01,9.223917e-01,-0.481887,-5.328378e-01,-2.296787e-01,-5.074783e-01,8.142748e-01,-4.904493e-01,8.235563e-01,-8.235563e-01,-2.714484e-01,9.978237e-01,-8.690559e-01
75%,6.883185e-01,4.109089e-01,0.623835,-5.513413e-01,7.541571e-01,5.747115e-01,-5.159524e-01,5.159524e-01,9.223917e-01,-0.481887,-5.328378e-01,-2.296787e-01,-5.074783e-01,8.142748e-01,-4.904493e-01,8.235563e-01,1.214246e+00,-2.714484e-01,9.978237e-01,1.150674e+00
max,2.491558e+00,3.653439e+00,3.697252,1.813758e+00,2.561874e+00,4.983762e+00,1.938163e+00,5.159524e-01,9.223917e-01,2.075177,1.876744e+00,4.353909e+00,1.970528e+00,8.142748e-01,2.038947e+00,8.235563e-01,1.214246e+00,3.683942e+00,9.978237e-01,1.150674e+00


**3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

**4. Обучите модель логистической регрессии с параметрами по умолчанию.**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# Предсказания целевой переменной:
y_pred_lr = model_lr.predict(X_test)

**5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:**

**`cross_validate(…, cv=10, scoring=['accuracy','recall','precision','f1'])`**

Т.к. тестовая выборка используется для итогового оценивания работы модели, то ко всему множеству объектов тестовой выборки вместо косс-валидации будем применять отдельные метрики.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

Метрики на обучающей выборке:

In [ ]:
metrics_lr_train = cross_validate(model_lr, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
metrics_lr_train

{'fit_time': array([0.00859499, 0.006634  , 0.0065763 , 0.00681543, 0.00651479,
        0.00639439, 0.00695848, 0.00685883, 0.0065217 , 0.00651145]),
 'score_time': array([0.00846052, 0.0104208 , 0.00821447, 0.00800443, 0.00797391,
        0.00781441, 0.00797081, 0.0082407 , 0.00786924, 0.00799131]),
 'test_accuracy': array([0.87837838, 0.89189189, 0.86486486, 0.83783784, 0.83561644,
        0.84931507, 0.83561644, 0.83561644, 0.84931507, 0.91780822]),
 'test_recall': array([0.9047619 , 0.88095238, 0.83333333, 0.88095238, 0.87804878,
        0.85365854, 0.92682927, 0.90243902, 0.87804878, 0.90243902]),
 'test_precision': array([0.88372093, 0.925     , 0.92105263, 0.84090909, 0.8372093 ,
        0.875     , 0.80851064, 0.82222222, 0.85714286, 0.94871795]),
 'test_f1': array([0.89411765, 0.90243902, 0.875     , 0.86046512, 0.85714286,
        0.86419753, 0.86363636, 0.86046512, 0.86746988, 0.925     ])}

Метрики тестовой выборки:

In [ ]:
metrics_lr_test = {'Accuracy' : accuracy_score(y_test, y_pred_lr),
                   'Recall' : recall_score(y_test, y_pred_lr),
                   'Precision' : precision_score(y_test, y_pred_lr),
                   'F1' : f1_score(y_test, y_pred_lr)}
metrics_lr_test

{'Accuracy': 0.8586956521739131,
 'Recall': 0.8829787234042553,
 'Precision': 0.8469387755102041,
 'F1': 0.8645833333333334}

Для удобства сравнения моделей и методов оптимизации, метрики будем собирать в датафрейм df_metrics.

Т.к. решается задача наличия/отсутствия болезни (классификации), то метрика accuracy не очень показательна. Оставим метрики Racall, Precision и F1.

Метрики обучающей выборки исходного датасета в датафрейме df_metrics обозначены как 'train' и представляют собой средние значения метрик тестовых подвыборок, рассчитанных при кросс-валидации.

Метрики тестовой выборки исходного датасета обозначены 'test' и рассчитаны без использования кросс-валидации.



In [ ]:
df_metrics = pd.DataFrame({'Model':['LogReg'], 'Optimization':['No'],
                           'Recall_CV(train)':[np.mean(metrics_lr_train['test_recall'])],
                           'Precision_CV(train)':[np.mean(metrics_lr_train['test_precision'])],
                           'F1_CV(train)':[np.mean(metrics_lr_train['test_f1'])],
                           'Recall(test)':[metrics_lr_test['Recall']],
                           'Precision(test)':[metrics_lr_test['Precision']],
                           'F1(test)':[metrics_lr_test['F1']]})

df_metrics

,Model,Optimization,Recall_CV(train),Precision_CV(train),F1_CV(train),Recall(test),Precision(test),F1(test)
0,LogReg,No,0.884146,0.871949,0.876993,0.882979,0.846939,0.864583


**6. Оптимизируйте 3-4 параметра модели:**

  **d) Повторите п. 5 после каждого итогового изменения параметров.**

  **a) Используйте GridSearchCV.**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Гиперпараметры логистической регрессии и их значения по умолчанию:
LogisticRegression().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

С помощью GridSearchCV выполним оптимизацию 'penalty', 'solver' и 'C'.

In [ ]:
penalties = ['l2', None]
solvers = ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
Cs = np.logspace(-4, 4, 20)

In [ ]:
param_grid = dict(solver=solvers, penalty=penalties, C=Cs)
print(param_grid)

{'solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga'], 'penalty': ['l2', None], 'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])}


In [ ]:
grid = GridSearchCV(model_lr, param_grid, cv=10)

In [ ]:
grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'penalty': ['l2', None],
                         'solver': ['lbfgs', 'newton-cg', 'newton-cholesky',
                                    'sag', 'saga']})

In [ ]:
# Лучшие параметры, полученные с помощью GridSearch:
p_GS = grid.best_params_
p_GS

{'C': 0.012742749857031334, 'penalty': 'l2', 'solver': 'lbfgs'}

Обучение лог.регрессии с гиперпараметрами, подобранными GridSearchCV, и ее метрики:

In [ ]:
# Train-выборка:
model_lr_GS = LogisticRegression(penalty=p_GS['penalty'], solver=p_GS['solver'], C=p_GS['C'])
model_lr_GS.fit(X_train, y_train)
y_pred_lr_GS = model_lr_GS.predict(X_test)

metrics_lr_GS_train = cross_validate(model_lr_GS, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
print(metrics_lr_GS_train['test_recall'])
metrics_lr_GS_train

[0.95238095 0.88095238 0.85714286 0.92857143 0.92682927 0.87804878
 0.92682927 0.90243902 0.90243902 0.90243902]


{'fit_time': array([0.01798964, 0.0165813 , 0.01838255, 0.01658797, 0.01657581,
        0.01509047, 0.01745677, 0.01381326, 0.01602888, 0.01440263]),
 'score_time': array([0.01499796, 0.01140237, 0.01561475, 0.01649189, 0.01836276,
        0.01848769, 0.01951456, 0.01814651, 0.01782513, 0.0178597 ]),
 'test_accuracy': array([0.91891892, 0.89189189, 0.85135135, 0.85135135, 0.8630137 ,
        0.84931507, 0.84931507, 0.84931507, 0.8630137 , 0.91780822]),
 'test_recall': array([0.95238095, 0.88095238, 0.85714286, 0.92857143, 0.92682927,
        0.87804878, 0.92682927, 0.90243902, 0.90243902, 0.90243902]),
 'test_precision': array([0.90909091, 0.925     , 0.87804878, 0.82978723, 0.84444444,
        0.85714286, 0.82608696, 0.84090909, 0.86046512, 0.94871795]),
 'test_f1': array([0.93023256, 0.90243902, 0.86746988, 0.87640449, 0.88372093,
        0.86746988, 0.87356322, 0.87058824, 0.88095238, 0.925     ])}

In [ ]:
# Test-выборка:
metrics_lr_GS_test = {'Accuracy' : accuracy_score(y_test, y_pred_lr_GS),
                   'Recall' : recall_score(y_test, y_pred_lr_GS),
                   'Precision' : precision_score(y_test, y_pred_lr_GS),
                   'F1' : f1_score(y_test, y_pred_lr_GS)}
metrics_lr_GS_test

{'Accuracy': 0.8641304347826086,
 'Recall': 0.8936170212765957,
 'Precision': 0.8484848484848485,
 'F1': 0.8704663212435233}

In [ ]:
# Дополнение датафрейма с метриками новыми данными:
df_metrics.loc[len(df_metrics)] = ['LogReg', 'GreadSearchCV',
                                                 np.mean(metrics_lr_GS_train['test_recall']),
                                                  np.mean(metrics_lr_GS_train['test_precision']),
                                                  np.mean(metrics_lr_GS_train['test_f1']),
                                                  metrics_lr_GS_test['Recall'],
                                                  metrics_lr_GS_test['Precision'],
                                                  metrics_lr_GS_test['F1']]

df_metrics

,Model,Optimization,Recall_CV(train),Precision_CV(train),F1_CV(train),Recall(test),Precision(test),F1(test)
0,LogReg,No,0.884146,0.871949,0.876993,0.882979,0.846939,0.864583
1,LogReg,GreadSearchCV,0.905807,0.871969,0.887784,0.893617,0.848485,0.870466


  **b) Используйте RandomizedSearchCV.**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Сетку значений гиперпараметров оставим ту же самую.

In [ ]:
r_search = RandomizedSearchCV(model_lr, param_grid, cv=10, n_iter=50, random_state=9)

In [ ]:
r_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and

RandomizedSearchCV(cv=10, estimator=LogisticRegression(), n_iter=50,
                   param_distributions={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                                        'penalty': ['l2', None],
                                        'solver': ['lbfgs', 'newton-cg',
                                                   'newton-cholesky', 'sag',
                                                   'saga']},
                   random_state=9)

In [ ]:
p_RS = r_search.best_params_
p_RS

{'solver': 'saga', 'penalty': 'l2', 'C': 0.012742749857031334}

Обучение лог.регрессии с гиперпараметрами, подобранными RandomizedSearchCV, и ее метрики:

In [ ]:
# Train-выборка:
model_lr_RS = LogisticRegression(penalty=p_RS['penalty'], solver=p_RS['solver'], C=p_RS['C'])
model_lr_RS.fit(X_train, y_train)
y_pred_lr_RS = model_lr_RS.predict(X_test)

metrics_lr_RS_train = cross_validate(model_lr, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
metrics_lr_RS_train

{'fit_time': array([0.01395488, 0.00904846, 0.006598  , 0.00687242, 0.00706625,
        0.00771451, 0.00702024, 0.00707626, 0.00694108, 0.00662017]),
 'score_time': array([0.00806761, 0.00841212, 0.01004124, 0.00804186, 0.00834489,
        0.00807405, 0.00825262, 0.00810981, 0.00810528, 0.00845194]),
 'test_accuracy': array([0.87837838, 0.89189189, 0.86486486, 0.83783784, 0.83561644,
        0.84931507, 0.83561644, 0.83561644, 0.84931507, 0.91780822]),
 'test_recall': array([0.9047619 , 0.88095238, 0.83333333, 0.88095238, 0.87804878,
        0.85365854, 0.92682927, 0.90243902, 0.87804878, 0.90243902]),
 'test_precision': array([0.88372093, 0.925     , 0.92105263, 0.84090909, 0.8372093 ,
        0.875     , 0.80851064, 0.82222222, 0.85714286, 0.94871795]),
 'test_f1': array([0.89411765, 0.90243902, 0.875     , 0.86046512, 0.85714286,
        0.86419753, 0.86363636, 0.86046512, 0.86746988, 0.925     ])}

In [ ]:
# Test-выборка:
metrics_lr_RS_test = {'Accuracy' : accuracy_score(y_test, y_pred_lr_RS),
                   'Recall' : recall_score(y_test, y_pred_lr_RS),
                   'Precision' : precision_score(y_test, y_pred_lr_RS),
                   'F1' : f1_score(y_test, y_pred_lr_RS)}
metrics_lr_RS_test

{'Accuracy': 0.8641304347826086,
 'Recall': 0.8936170212765957,
 'Precision': 0.8484848484848485,
 'F1': 0.8704663212435233}

In [ ]:
# Дополнение датафрейма с метриками новыми данными:
df_metrics.loc[len(df_metrics)] = ['LogReg', 'RandomizedSearchCV',
                                                 np.mean(metrics_lr_RS_train['test_recall']),
                                                  np.mean(metrics_lr_RS_train['test_precision']),
                                                  np.mean(metrics_lr_RS_train['test_f1']),
                                                  metrics_lr_RS_test['Recall'],
                                                  metrics_lr_RS_test['Precision'],
                                                  metrics_lr_RS_test['F1']]

df_metrics

,Model,Optimization,Recall_CV(train),Precision_CV(train),F1_CV(train),Recall(test),Precision(test),F1(test)
0,LogReg,No,0.884146,0.871949,0.876993,0.882979,0.846939,0.864583
1,LogReg,GreadSearchCV,0.905807,0.871969,0.887784,0.893617,0.848485,0.870466
2,LogReg,RandomizedSearchCV,0.884146,0.871949,0.876993,0.893617,0.848485,0.870466


  **c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.**

В качестве моделей выберем RandomForestClassifier и SVC.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

Оптимизируемые параметры для случайного леса: 'criterion', 'max_depth' и 'min_samples_leaf'.

In [ ]:
SVC().get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

Оптимизируемые параметры для опорных векторов: 'C' и 'kernel'.

In [ ]:
estimators = [RandomForestClassifier(), SVC()]
grid_RF = dict(criterion=['gini', 'entropy', 'log_loss'], max_depth=[2, 20, 100], min_samples_leaf=[1, 10, 50])
grid_SVC = dict(C=np.logspace(-4, 4, 20), kernel=['linear', 'poly', 'rbf', 'sigmoid'])

param_grids = [grid_RF, grid_SVC]

In [ ]:
for estim, grid in zip(estimators, param_grids):
  r_search_i = RandomizedSearchCV(estim, grid, random_state=9)
  r_search_i.fit(X_train, y_train)
  best_p = r_search_i.best_params_
  print(best_p)

{'min_samples_leaf': 10, 'max_depth': 100, 'criterion': 'gini'}
{'kernel': 'sigmoid', 'C': 0.08858667904100823}


Обучим модели с полученными оптимальными значениями гиперпараметров.

Модель "Случайный лес" и ее метрики на обучающем и тестовом множестве:

In [ ]:
model_RF = RandomForestClassifier(min_samples_leaf=1, max_depth=100, criterion='gini', random_state=9)
model_RF.fit(X_train, y_train)

RandomForestClassifier(max_depth=100, random_state=9)

In [ ]:
metrics_RF_train = cross_validate(model_RF, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
metrics_RF_train

{'fit_time': array([0.20168161, 0.20525718, 0.19581938, 0.20397258, 0.19647098,
        0.20792603, 0.1905694 , 0.20422697, 0.19162941, 0.20543981]),
 'score_time': array([0.02033806, 0.02227449, 0.01786757, 0.01979613, 0.01829481,
        0.01597285, 0.01639438, 0.01718926, 0.01638365, 0.01640606]),
 'test_accuracy': array([0.91891892, 0.89189189, 0.87837838, 0.87837838, 0.82191781,
        0.8630137 , 0.82191781, 0.8630137 , 0.8630137 , 0.90410959]),
 'test_recall': array([0.95238095, 0.92857143, 0.92857143, 0.92857143, 0.87804878,
        0.87804878, 0.90243902, 0.87804878, 0.90243902, 0.90243902]),
 'test_precision': array([0.90909091, 0.88636364, 0.86666667, 0.86666667, 0.81818182,
        0.87804878, 0.80434783, 0.87804878, 0.86046512, 0.925     ]),
 'test_f1': array([0.93023256, 0.90697674, 0.89655172, 0.89655172, 0.84705882,
        0.87804878, 0.85057471, 0.87804878, 0.88095238, 0.91358025])}

In [ ]:
metrics_RF_test = {'Accuracy' : accuracy_score(y_test, model_RF.predict(X_test)),
                   'Recall' : recall_score(y_test, model_RF.predict(X_test)),
                   'Precision' : precision_score(y_test, model_RF.predict(X_test)),
                   'F1' : f1_score(y_test, model_RF.predict(X_test))}
metrics_RF_test

{'Accuracy': 0.8804347826086957,
 'Recall': 0.9361702127659575,
 'Precision': 0.8461538461538461,
 'F1': 0.8888888888888888}

In [ ]:
# Дополнение датафрейма с метриками новыми данными:
df_metrics.loc[len(df_metrics)] = ['RandomForestClassifier', 'RandomizedSearchCV',
                                                 np.mean(metrics_RF_train['test_recall']),
                                                  np.mean(metrics_RF_train['test_precision']),
                                                  np.mean(metrics_RF_train['test_f1']),
                                                  metrics_RF_test  ['Recall'],
                                                  metrics_RF_test  ['Precision'],
                                                  metrics_RF_test  ['F1']]

df_metrics

,Model,Optimization,Recall_CV(train),Precision_CV(train),F1_CV(train),Recall(test),Precision(test),F1(test)
0,LogReg,No,0.884146,0.871949,0.876993,0.882979,0.846939,0.864583
1,LogReg,GreadSearchCV,0.905807,0.871969,0.887784,0.893617,0.848485,0.870466
2,LogReg,RandomizedSearchCV,0.884146,0.871949,0.876993,0.893617,0.848485,0.870466
3,RandomForestClassifier,RandomizedSearchCV,0.907956,0.869288,0.887858,0.936170,0.846154,0.888889


Модель опорных векторов и ее метрики на обучающем и тестовом множествах:

In [ ]:
model_SVC = SVC(kernel='sigmoid', C=0.08858667904100823)
model_SVC.fit(X_train, y_train)

SVC(C=0.08858667904100823, kernel='sigmoid')

In [ ]:
metrics_SVC_train = cross_validate(model_SVC, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
metrics_SVC_train

{'fit_time': array([0.02334785, 0.01918316, 0.02888608, 0.01826739, 0.01895452,
        0.02066588, 0.02449679, 0.01896882, 0.01904368, 0.02068782]),
 'score_time': array([0.00819278, 0.00790048, 0.0081625 , 0.00782084, 0.00785971,
        0.00872016, 0.00803328, 0.00777555, 0.00861335, 0.00791478]),
 'test_accuracy': array([0.90540541, 0.87837838, 0.85135135, 0.86486486, 0.78082192,
        0.87671233, 0.82191781, 0.84931507, 0.84931507, 0.90410959]),
 'test_recall': array([0.97619048, 0.88095238, 0.85714286, 0.9047619 , 0.82926829,
        0.92682927, 0.90243902, 0.90243902, 0.90243902, 0.87804878]),
 'test_precision': array([0.87234043, 0.90243902, 0.87804878, 0.86363636, 0.79069767,
        0.86363636, 0.80434783, 0.84090909, 0.84090909, 0.94736842]),
 'test_f1': array([0.92134831, 0.89156627, 0.86746988, 0.88372093, 0.80952381,
        0.89411765, 0.85057471, 0.87058824, 0.87058824, 0.91139241])}

In [ ]:
metrics_SVC_test = {'Accuracy' : accuracy_score(y_test, model_SVC.predict(X_test)),
                   'Recall' : recall_score(y_test, model_SVC.predict(X_test)),
                   'Precision' : precision_score(y_test, model_SVC.predict(X_test)),
                   'F1' : f1_score(y_test, model_SVC.predict(X_test))}
metrics_SVC_test

{'Accuracy': 0.8804347826086957,
 'Recall': 0.9148936170212766,
 'Precision': 0.86,
 'F1': 0.8865979381443299}

In [ ]:
# Дополнение датафрейма с метриками новыми данными:
df_metrics.loc[len(df_metrics)] = ['SVC', 'RandomizedSearchCV',
                                                 np.mean(metrics_SVC_train['test_recall']),
                                                  np.mean(metrics_SVC_train['test_precision']),
                                                  np.mean(metrics_SVC_train['test_f1']),
                                                  metrics_SVC_test  ['Recall'],
                                                  metrics_SVC_test  ['Precision'],
                                                  metrics_SVC_test  ['F1']]

**7. Сформулируйте выводы по проделанной работе:**

  **a) Сравните метрики построенных моделей.**

In [ ]:
# Датафрейм метрик всех исследованных моделей и методов:
df_metrics

,Model,Optimization,Recall_CV(train),Precision_CV(train),F1_CV(train),Recall(test),Precision(test),F1(test)
0,LogReg,No,0.884146,0.871949,0.876993,0.882979,0.846939,0.864583
1,LogReg,GreadSearchCV,0.905807,0.871969,0.887784,0.893617,0.848485,0.870466
2,LogReg,RandomizedSearchCV,0.884146,0.871949,0.876993,0.893617,0.848485,0.870466
3,RandomForestClassifier,RandomizedSearchCV,0.907956,0.869288,0.887858,0.936170,0.846154,0.888889
4,SVC,RandomizedSearchCV,0.896051,0.860433,0.877089,0.914894,0.860000,0.886598


В целом метрики на тестовой выборке не сильно отличаются от метрик на обучающей, что говорит о том, что большинство моделей не переобучались (одним из оптимизируемых параметров был 'penalty').

Учитывая особенности целевой переменной (наличие болезни), самой показательной метрикой является Recall. Данная метрика на тестовой выборке показала самые высокие значения для оптимизированной с помощью RandomizedSearchCV модели Random Forest (0,936) и SVC (0,915), самое низкое значение - для логистической регрессии с гиперпараметрами по умолчанию (0,88).

Оптимизация логистической регрессии с помощью GreadSearchCV	 и RandomizedSearchCV на тесте дала одинаковые метрики.

В итоге для данной задачи лучше других из исследуемых моделей показали себя модели Random Forest с RandomizedSearchCV (самый высокий Recall на тесте) и SVC с RandomizedSearchCV (Recall второй по величине, Precision и F1 на тесте и трейне почти одинаковые, т.е. модель самая стабильная).

  **b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».**

Значения метрик из ДЗ по теме "Ансамблирование" представлены ниже. Лучшие значения метрик были у Random Forest и Stacking.

В данной работе оптимизация применение RandomizedSearchCV с Random Forest дало похожие результаты.   

DECISION TREE               
                
                precision    recall  f1-score   support

       Normal       0.78      0.76      0.77        90
       Disease      0.77      0.80      0.79        94

     accuracy                           0.78       184
    macro avg       0.78      0.78      0.78       184
    weighted avg    0.78      0.78      0.78       184


RANDOM FOREST

                precision    recall  f1-score   support

       Normal       0.93      0.82      0.87        90
       Disease      0.85      0.94      0.89        94

     accuracy                           0.88       184
    macro avg       0.89      0.88      0.88       184
    weighted avg    0.88      0.88      0.88       184

BAGGING

                precision    recall  f1-score   support

       Normal       0.49      1.00      0.66        90
       Disease      0.00      0.00      0.00        94

     accuracy                           0.49       184
    macro avg       0.24      0.50      0.33       184
    weighted avg    0.24      0.49      0.32       184

STACKING

                precision    recall  f1-score   support

       Normal       0.92      0.81      0.86        90
       Disease      0.84      0.94      0.88        94

     accuracy                           0.88       184
    macro avg       0.88      0.87      0.87       184
    weighted avg    0.88      0.88      0.87       184